<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/bert/tf_bert_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT from scratch

## Intro

In [ ]:
#@title ## Import packages
import os
import shutil
from dataclasses import dataclass
from google.colab import auth, data_table
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.preprocessing import text_dataset_from_directory 

tf.get_logger().setLevel("ERROR")

print("tensorflow", tf.__version__)

tensorflow 2.4.0


## 1. Configuration

In [ ]:
@dataclass
class Config:
    MAX_LEN = 256
    BATCH_SIZE = 32
    LR = 0.001
    VOCAB_SIZE = 30000
    EMBED_DIM = 128
    NUM_HEAD = 8
    FF_DIM = 128
    NUM_LAYERS = 1
    DS_URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz" 
    TRAIN_DIR = None
    TEST_DIR = None
    VALIDATION_SPLIT = 0.2
    SEED = 42

config = Config()

## 2. Download Dataset - Large Movie Review

We use [Stanford’s Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) as the dataset for sentiment analysis. This dataset is divided into two datasets for training and testing purposes, each containing 25,000 movie reviews downloaded from IMDb. In each dataset, the number of comments labeled as “positive” and “negative” is equal.


In [ ]:
dataset = tf.keras.utils.get_file(
    "aclImdb.tar.gz", config.DS_URL, untar=True, cache_dir=".", cache_subdir=""
)
dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")
config.TRAIN_DIR = os.path.join(dataset_dir, "train")
config.TEST_DIR = os.path.join(dataset_dir, "test")
print("Train dataset:", config.TRAIN_DIR)
print("Train dataset:", config.TEST_DIR )

# remove unused folders from train folder
remove_dir = os.path.join(config.TRAIN_DIR, "unsup")
shutil.rmtree(remove_dir)

Train dataset: ./aclImdb/train
Train dataset: ./aclImdb/test


The IMDB dataset has already been divided into train and test, but it lacks a validation set. Let's create a validation split using a 80:20 split.

> Tensorflow Datasource requires to define a random seed or to pass shuffle=False so that the validation and training splits have no overlap. **We will pass a random seed**.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def optimize_dataset(dataset):
    return dataset.cache().prefetch(buffer_size=AUTOTUNE)

train_ds = text_dataset_from_directory(
    config.TRAIN_DIR,
    subset="training",
    batch_size=config.BATCH_SIZE,
    validation_split=config.VALIDATION_SPLIT,
    seed=config.SEED,
)
class_names = train_ds.class_names
train_ds = optimize_dataset(train_ds)

val_ds = text_dataset_from_directory(
    config.TRAIN_DIR,
    subset="validation",
    batch_size=config.BATCH_SIZE,
    validation_split=config.VALIDATION_SPLIT,
    seed=config.SEED,
)
val_ds = optimize_dataset(val_ds)

test_ds = text_dataset_from_directory(
    config.TEST_DIR,
    batch_size=config.BATCH_SIZE
)
test_ds = optimize_dataset(test_ds)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [ ]:
print("Loaded classes:", class_names)

Loaded classes: ['neg', 'pos']


In [ ]:
text_batch, label_batch = next(train_ds.as_numpy_iterator())
df = pd.DataFrame({"text": text_batch, "label": label_batch})
data_table.DataTable(df, include_index=False, num_rows_per_page=3, )

,text,label
0,"b""Having seen most of Ringo Lam's films, I can...",1
1,b'Caution: May contain spoilers...<br /><br />...,1
2,"b""from the view of a NASCAR Maniac like I am, ...",1
3,"b'When it first came out, this work by the Mey...",1
4,"b""I thought that this was an absolutely charmi...",1
5,"b""The filming is pleasant and the environment ...",1
6,"b""This is one of those cheaply made TV Movies ...",0
7,b'Great movie - especially the music - Etta Ja...,0
8,b'This film is not even worth walking to the m...,0
9,"b""For me, this movie just seemed to fall on it...",0


## 3. Preprocessing dataset